In [ ]:
# Install selenium
# !pip install selenium

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
import time
from selenium.webdriver.common.by import By
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
import json
import os

scrapper_config = open('../../config/scrapper_config.json')
scrapper_config = json.load(scrapper_config)


driver = webdriver.Chrome()
driver.get(scrapper_config["url"])

# Enter From Date
time.sleep(3)
date_input = driver.find_element(By.ID,"date_received-from")
date_input.send_keys(scrapper_config["from_date"])

# Enter To Date
time.sleep(3)
date_input = driver.find_element(By.ID,"date_received-through")
date_input.send_keys(scrapper_config["to_date"])

# Hit Enter
time.sleep(3)
date_input.send_keys(Keys.ENTER)

# Click Export
time.sleep(3)
button_element = driver.find_element(By.XPATH, '//*[@id="search-summary"]/div[2]/h3/button[1]')
button_element.click()

# Choose Filtered Dataset
time.sleep(3)
button_element = driver.find_element(By.XPATH, '/html/body/div[5]/div/div/section/div[2]/div[3]/div[2]/div[1]/label')
button_element.click()

## Click Start Export
time.sleep(3)
button_element = driver.find_element(By.XPATH, '/html/body/div[5]/div/div/section/div[3]/button[1]')
button_element.click()

## Click Close
time.sleep(3)
button_element = driver.find_element(By.XPATH, '/html/body/div[5]/div/div/section/div[1]/button')
button_element.click()

## Close Chrome Window After Sleep Time
time.sleep(scrapper_config["windowshuttime"])
driver.quit()


def valid_files():
    temp=[]
    all_files = os.listdir(scrapper_config["download_dir"])
    for file in all_files:
        if "complaints" in file and ".csv"==file[-4:]:
            os.rename(scrapper_config["download_dir"]+file,scrapper_config["scrapped_dir"]+str(round(time.time()))+"_"+file)
            temp.append(str(round(time.time()))+"_"+file)
    return temp
            
valid_files = valid_files()

scrapped_file = os.listdir(scrapper_config["scrapped_dir"])

if len(valid_files)==0:
    print("There are no files to move")
elif set(valid_files).issubset(set(scrapped_file)): 
    print(f"{len(valid_files)} Files are moved succefully")
else:
    print("Something went wrong here")

In [ ]:
# Install kafka
# !pip install kafka-python

from kafka import KafkaProducer
import json
import csv
import os
import time

time.sleep(5)
kafka_config = open('../../config/kafka_config.json')
kafka_config = json.load(kafka_config)

scrapper_config = open('../../config/scrapper_config.json')
scrapper_config = json.load(scrapper_config)

scrapped_file = os.listdir(scrapper_config["scrapped_dir"])

producer = KafkaProducer(bootstrap_servers=kafka_config["bootstrap_servers_IP_port"], value_serializer=lambda v: json.dumps(v).encode('utf-8'),)

for file in scrapped_file:
    print(f"============ Sending filename={file}=================")
    with open(scrapper_config["scrapped_dir"]+file, 'r') as datafile:
            csv_reader = csv.DictReader(datafile)
            for row in csv_reader:
                producer.send(kafka_config["kafka_topic"], value=row)
            print(f"=============File sent successfully filename ={file}=============")

            
for file in scrapped_file:
    os.remove(scrapper_config["scrapped_dir"]+file)
    print(f"=============File deleted successfully filename ={file}=============")